In [1]:
import GPyOpt
import numpy as np
import pandas as pd
import re

In [24]:
KINIT_USERNAME = 'oalenkin'
KINIT_PASSWD = 'Ubivator94'
N_EVENTS = 2000

# Optimization space

In [3]:
min_dist = 3.6

space = [{'name': 'pitch', 'type': 'continuous', 'domain': (min_dist, min_dist)},\
         {'name': 'yoffset_layer', 'type': 'continuous', 'domain': (min_dist/2, min_dist)},\
         {'name': 'yoffset_plane', 'type': 'continuous', 'domain': (min_dist*0.25, min_dist*1.25)},\
         {'name': 'zshift_layer', 'type': 'continuous', 'domain': (1.6, 2.6)},\
         {'name': 'zshift_plane', 'type': 'continuous', 'domain': (3.8, 6.8)},\
         {'name': 'zshift_view', 'type': 'continuous', 'domain': (10, 10)},\
         {'name': 'alpha', 'type': 'discrete', 'domain': (5, 5)}]

constraints = [{'name': 'constr_1', 'constrain': '-(x[:,0]-x[:,1])**2-x[:,3]**2+2**2'},\
               {'name': 'constr_2', 'constrain': '-(x[:,1]-x[:,2])**2-(x[:,3]-x[:,4])**2+2**2'},\
               {'name': 'constr_3', 'constrain': 'x[:,3]+x[:,4]+2-x[:,5]'}]

In [4]:
feasible_region = GPyOpt.Design_space(space=space, constraints=constraints)

np.random.seed(42)

# Import skygrid client

In [3]:
import time
import json

from disneylandClient import (
    new_client,
    Job,
    RequestWithId,
)

In [4]:
STATUS_IN_PROCESS = set([
    Job.PENDING,
    Job.PULLED,
    Job.RUNNING,
])
STATUS_FINAL = set([
    Job.COMPLETED,
    Job.FAILED,
])

In [5]:
def return_descriptor(point):
    
    pitch, yoffset_layer, yoffset_plane, zshift_layer, zshift_plane, zshift_view, alpha = point
    
    logining = "sh -lc 'echo "+KINIT_PASSWD+" | kinit "+KINIT_USERNAME+"; "
    sourcing = "source /opt/FairShipRun/config.sh; "
    simulation = "python $SHIPOPT/code/objective.py --pitch "+str(pitch)+" --yoffset_layer "+str(yoffset_layer)+\
              " --yoffset_plane "+str(yoffset_plane)+" --zshift_layer "+str(zshift_layer)+" --zshift_plane "+\
              str(zshift_plane)+" --zshift_view "+str(zshift_view)+" --alpha "+str(int(alpha))+\
              " --nEvents "+str(N_EVENTS)+" --output /output/output.txt'"
    cmd = logining+sourcing+simulation

    descriptor = {
        "input": [],

        "container": {
            "workdir": "",
            "name": "oleg94/worker_layer:local_importing",
            "cpu_needed": 1,
            "max_memoryMB": 4096,
            "min_memoryMB": 1024,
            "cmd": cmd,
        },

        "required_outputs": {
            "output_uri": "none:",
            "file_contents": [
                {"file": "output.txt", "to_variable": "out"}
            ]
        }
    }
    
    return descriptor

# Initial design

In [18]:
n_estimators = 100
n_initial_design = 100

In [19]:
initial_design = GPyOpt.experiment_design.initial_design('random', feasible_region, n_initial_design)

In [10]:
initial_objective = np.zeros(n_initial_design)

In [11]:
stub = new_client()
jobs = []

In [12]:
init_d_i = 0

for epoch in range(n_initial_design // n_estimators):
    
    print("EPOCH #"+str(epoch)+" started.")
    
    epoch_jobs = [0] * n_estimators
    
    for k in range(n_estimators):
        descriptor = return_descriptor(initial_design[init_d_i])
        init_d_i += 1
        epoch_jobs[k] = Job(input=json.dumps(descriptor), kind="docker")
        epoch_jobs[k] = stub.CreateJob(epoch_jobs[k])
        print(k, " pushed")
    
    prev_number_of_finished_jobs = 0
    prev_number_of_running_jobs = 0
    prev_number_of_pending_jobs = 0
    
    while True:
        for k in range(n_estimators):
            epoch_jobs[k] = stub.GetJob(RequestWithId(id=epoch_jobs[k].id))
        
        number_of_finished_jobs = 0
        number_of_running_jobs = 0
        number_of_pending_jobs = 0
        for k in range(n_estimators):
            if epoch_jobs[k].status in STATUS_FINAL:
                number_of_finished_jobs += 1
            if epoch_jobs[k].status == Job.PENDING:
                number_of_pending_jobs += 1
            if epoch_jobs[k].status == Job.RUNNING:
                number_of_running_jobs += 1
                
        if (number_of_finished_jobs != prev_number_of_finished_jobs) or (prev_number_of_running_jobs != number_of_running_jobs) or (prev_number_of_pending_jobs != number_of_pending_jobs):
            print("Finished jobs: "+str(number_of_finished_jobs)+\
                  " Running jobs: "+str(number_of_running_jobs)+\
                  " Pending jobs: "+str(number_of_pending_jobs))
            prev_number_of_finished_jobs = number_of_finished_jobs
            prev_number_of_running_jobs = number_of_running_jobs
            prev_number_of_pending_jobs = number_of_pending_jobs
            
        if number_of_finished_jobs == n_estimators:
            break
        time.sleep(120)
    
    jobs += epoch_jobs

EPOCH #0 started.
0  pushed
1  pushed
2  pushed
3  pushed
4  pushed
5  pushed
6  pushed
7  pushed
8  pushed
9  pushed
10  pushed
11  pushed
12  pushed
13  pushed
14  pushed
15  pushed
16  pushed
17  pushed
18  pushed
19  pushed
20  pushed
21  pushed
22  pushed
23  pushed
24  pushed
25  pushed
26  pushed
27  pushed
28  pushed
29  pushed
30  pushed
31  pushed
32  pushed
33  pushed
34  pushed
35  pushed
36  pushed
37  pushed
38  pushed
39  pushed
40  pushed
41  pushed
42  pushed
43  pushed
44  pushed
45  pushed
46  pushed
47  pushed
48  pushed
49  pushed
50  pushed
51  pushed
52  pushed
53  pushed
54  pushed
55  pushed
56  pushed
57  pushed
58  pushed
59  pushed
60  pushed
61  pushed
62  pushed
63  pushed
64  pushed
65  pushed
66  pushed
67  pushed
68  pushed
69  pushed
70  pushed
71  pushed
72  pushed
73  pushed
74  pushed
75  pushed
76  pushed
77  pushed
78  pushed
79  pushed
80  pushed
81  pushed
82  pushed
83  pushed
84  pushed
85  pushed
86  pushed
87  pushed
88  pushed
89  pushed
90

In [13]:
df_init_design = pd.DataFrame(initial_design, columns=['pitch', 'yoffset_layer', 'yoffset_plane', 'zshift_layer', 'zshift_plane', 'zshift_view', 'alpha'])
df_init_design['objective'] = [float(re.sub('[\[\]"variable:out=]', '', job.output)) if re.sub('[\[\]"variable:out=]', '', job.output) else np.nan for job in jobs]

In [14]:
df_init_design.to_csv('../observations/observations.csv', index=False)

# Main part of optimization

In [8]:
df_init_design = pd.read_csv('../observations/observations.csv')
n_estimators = 60
n_epochs = 500
stub = new_client()

In [ ]:
for epoch in range(n_epochs):
    
        print("EPOCH #"+str(epoch)+" started.")
    
#         epoch_jobs = [0] * n_estimators
        
#         pending_X = []

#         for k in range(n_estimators):
            
#             step_X = df_init_design[df_init_design.columns[:-1]].values
#             #because we want to maximize
#             step_Y = -df_init_design[df_init_design.columns[-1:]].values
#             ignored_X = step_X[np.isnan(step_Y.ravel())]
#             step_X = step_X[~np.isnan(step_Y.ravel())]
#             step_Y = step_Y[~np.isnan(step_Y.ravel())]
#             bo = GPyOpt.methods.BayesianOptimization(f=None, domain=space, constraints=constraints, X=step_X,\
#                                                      Y=step_Y, initial_design_numdata=100,\
#                                                      evaluator_type='local_penalization', batch_size=n_estimators)
#             new_point = bo.suggest_next_locations(pending_X=np.array(pending_X), ignored_X=ignored_X)[0]

#             descriptor = return_descriptor(new_point)
#             epoch_jobs[k] = Job(input=json.dumps(descriptor), kind="docker")
#             epoch_jobs[k] = stub.CreateJob(epoch_jobs[k])
            
#             pending_X += [new_point]


        step_X = df_init_design[df_init_design.columns[:-1]].values
        #because we want to maximize
        step_Y = -df_init_design[df_init_design.columns[-1:]].values
        ignored_X = step_X[np.isnan(step_Y.ravel())]
        step_X = step_X[~np.isnan(step_Y.ravel())]
        step_Y = step_Y[~np.isnan(step_Y.ravel())]
        bo = GPyOpt.methods.BayesianOptimization(f=None, domain=space, constraints=constraints, X=step_X,\
                                                 Y=step_Y, initial_design_numdata=100,\
                                                 evaluator_type='local_penalization', batch_size=n_estimators)
        
        pending_X = list(bo.suggest_next_locations(ignored_X=ignored_X))
        
        epoch_jobs = [0] * len(pending_X)
        for k, new_point in enumerate(pending_X):
            descriptor = return_descriptor(new_point)
            epoch_jobs[k] = Job(input=json.dumps(descriptor), kind="docker")
            epoch_jobs[k] = stub.CreateJob(epoch_jobs[k])
        
        prev_number_of_finished_jobs = 0
        prev_number_of_running_jobs = 0
        prev_number_of_pending_jobs = 0

        while True:
            for k in range(n_estimators):
                epoch_jobs[k] = stub.GetJob(RequestWithId(id=epoch_jobs[k].id))

            number_of_finished_jobs = 0
            number_of_running_jobs = 0
            number_of_pending_jobs = 0
            for k in range(n_estimators):
                if epoch_jobs[k].status in STATUS_FINAL:
                    number_of_finished_jobs += 1
                if epoch_jobs[k].status == Job.PENDING:
                    number_of_pending_jobs += 1
                if epoch_jobs[k].status == Job.RUNNING:
                    number_of_running_jobs += 1

            if (number_of_finished_jobs != prev_number_of_finished_jobs) or (prev_number_of_running_jobs != number_of_running_jobs) or (prev_number_of_pending_jobs != number_of_pending_jobs):
                print("Finished jobs: "+str(number_of_finished_jobs)+\
                      " Running jobs: "+str(number_of_running_jobs)+\
                      " Pending jobs: "+str(number_of_pending_jobs))
                prev_number_of_finished_jobs = number_of_finished_jobs
                prev_number_of_running_jobs = number_of_running_jobs
                prev_number_of_pending_jobs = number_of_pending_jobs

            if number_of_finished_jobs == n_estimators:
                break
            time.sleep(120)
            
        for k, point in enumerate(pending_X):
        
            value = float(re.sub('[\[\]"variable:out=]', '', epoch_jobs[k].output)) if re.sub('[\[\]"variable:out=]', '', epoch_jobs[k].output) else np.nan
        
            df_init_design.loc[len(df_init_design)] = list(point)+[value]
            df_init_design.to_csv('../observations/observations.csv', index=False)

EPOCH #0 started.
Finished jobs: 0 Running jobs: 33 Pending jobs: 25
Finished jobs: 0 Running jobs: 60 Pending jobs: 0


In [39]:
for k in range(n_estimators):
    epoch_jobs[k] = stub.GetJob(RequestWithId(id=epoch_jobs[k].id))

number_of_finished_jobs = 0
number_of_running_jobs = 0
number_of_pending_jobs = 0
for k in range(n_estimators):
    if epoch_jobs[k].status in STATUS_FINAL:
        number_of_finished_jobs += 1
    if epoch_jobs[k].status == Job.PENDING:
        number_of_pending_jobs += 1
    if epoch_jobs[k].status == Job.RUNNING:
        number_of_running_jobs += 1


print("Finished jobs: "+str(number_of_finished_jobs)+\
      " Running jobs: "+str(number_of_running_jobs)+\
      " Pending jobs: "+str(number_of_pending_jobs))

Finished jobs: 42 Running jobs: 18 Pending jobs: 0


In [40]:
for k, point in enumerate(pending_X):
        
    value = float(re.sub('[\[\]"variable:out=]', '', epoch_jobs[k].output)) if re.sub('[\[\]"variable:out=]', '', epoch_jobs[k].output) else np.nan
    df_init_design.loc[len(df_init_design)] = list(point)+[value]

In [41]:
df_init_design[~np.isnan(df_init_design.objective)].to_csv('../observations/observations.csv', index=False)

# Interesting points

In [25]:
design = [[3.6, 1.9 , 1.3, 1.6, 4.2, 10., 5.], [3.6, 1.9 , 1.3, 1.8, 4.2, 10., 5.],\
          [3.6, 1.8 , 1.2, 1.8, 4.4, 10., 5.], [3.6, 1.8 , 0.9, 1.8, 4.2, 10., 5.],\
          [3.6, 1.81362 , 0.96872, 1.98391, 5.93714, 10., 5.]] * 15

In [38]:
design = np.array(design)

In [39]:
objective = np.zeros(len(design))
stub = new_client()
jobs = []

In [40]:
for k in range(len(design)):
    descriptor = return_descriptor(design[k])
    jobs.append(stub.CreateJob(Job(input=json.dumps(descriptor), kind="docker")))
    print(k, " pushed")

prev_number_of_finished_jobs = 0
prev_number_of_running_jobs = 0
prev_number_of_pending_jobs = 0

while True:
    for k in range(len(design)):
        jobs[k] = stub.GetJob(RequestWithId(id=jobs[k].id))

    number_of_finished_jobs = 0
    number_of_running_jobs = 0
    number_of_pending_jobs = 0
    for k in range(len(design)):
        if jobs[k].status in STATUS_FINAL:
            number_of_finished_jobs += 1
        if jobs[k].status == Job.PENDING:
            number_of_pending_jobs += 1
        if jobs[k].status == Job.RUNNING:
            number_of_running_jobs += 1

    if (number_of_finished_jobs != prev_number_of_finished_jobs) or (prev_number_of_running_jobs != number_of_running_jobs) or (prev_number_of_pending_jobs != number_of_pending_jobs):
        print("Finished jobs: "+str(number_of_finished_jobs)+\
              " Running jobs: "+str(number_of_running_jobs)+\
              " Pending jobs: "+str(number_of_pending_jobs))
        prev_number_of_finished_jobs = number_of_finished_jobs
        prev_number_of_running_jobs = number_of_running_jobs
        prev_number_of_pending_jobs = number_of_pending_jobs

    if number_of_finished_jobs == len(design):
        break
    time.sleep(120)

0  pushed
1  pushed
2  pushed
3  pushed
4  pushed
5  pushed
6  pushed
7  pushed
8  pushed
9  pushed
10  pushed
11  pushed
12  pushed
13  pushed
14  pushed
15  pushed
16  pushed
17  pushed
18  pushed
19  pushed
20  pushed
21  pushed
22  pushed
23  pushed
24  pushed
25  pushed
26  pushed
27  pushed
28  pushed
29  pushed
30  pushed
31  pushed
32  pushed
33  pushed
34  pushed
35  pushed
36  pushed
37  pushed
38  pushed
39  pushed
40  pushed
41  pushed
42  pushed
43  pushed
44  pushed
45  pushed
46  pushed
47  pushed
48  pushed
49  pushed
50  pushed
51  pushed
52  pushed
53  pushed
54  pushed
55  pushed
56  pushed
57  pushed
58  pushed
59  pushed
60  pushed
61  pushed
62  pushed
63  pushed
64  pushed
65  pushed
66  pushed
67  pushed
68  pushed
69  pushed
70  pushed
71  pushed
72  pushed
73  pushed
74  pushed
Finished jobs: 0 Running jobs: 4 Pending jobs: 69
Finished jobs: 0 Running jobs: 75 Pending jobs: 0
Finished jobs: 5 Running jobs: 70 Pending jobs: 0
Finished jobs: 10 Running jobs: 65 

In [41]:
df_design = pd.DataFrame(design, columns=['pitch', 'yoffset_layer', 'yoffset_plane', 'zshift_layer', 'zshift_plane', 'zshift_view', 'alpha'])
df_design['objective'] = [float(re.sub('[\[\]"variable:out=]', '', job.output)) if re.sub('[\[\]"variable:out=]', '', job.output) else np.nan for job in jobs]

In [42]:
df_design

,pitch,yoffset_layer,yoffset_plane,zshift_layer,zshift_plane,zshift_view,alpha,objective
0,3.6,1.90000,1.30000,1.60000,4.20000,10.0,5.0,0.997093
1,3.6,1.90000,1.30000,1.80000,4.20000,10.0,5.0,1.000000
2,3.6,1.80000,1.20000,1.80000,4.40000,10.0,5.0,1.000000
3,3.6,1.80000,0.90000,1.80000,4.20000,10.0,5.0,1.000000
4,3.6,1.81362,0.96872,1.98391,5.93714,10.0,5.0,1.000000
5,3.6,1.90000,1.30000,1.60000,4.20000,10.0,5.0,1.000000
6,3.6,1.90000,1.30000,1.80000,4.20000,10.0,5.0,1.000000
7,3.6,1.80000,1.20000,1.80000,4.40000,10.0,5.0,1.000000
8,3.6,1.80000,0.90000,1.80000,4.20000,10.0,5.0,1.000000
9,3.6,1.81362,0.96872,1.98391,5.93714,10.0,5.0,1.000000


In [43]:
df3 = df_design.groupby(['yoffset_layer', 'yoffset_plane', 'zshift_layer', 'zshift_plane'], as_index=False)['objective'].mean()

In [37]:
df_design.to_csv('observations/interesting_points2.csv', index=False)

In [31]:
df_design.groupby(['yoffset_layer', 'yoffset_plane', 'zshift_layer', 'zshift_plane'], as_index=False)['objective'].mean()

,yoffset_layer,yoffset_plane,zshift_layer,zshift_plane,objective
0,1.80000,0.90000,1.80000,4.20000,0.999800
1,1.80000,1.20000,1.80000,4.40000,1.000000
2,1.81362,0.96872,1.98391,5.93714,0.999808
3,1.90000,1.30000,1.60000,4.20000,0.999045
4,1.90000,1.30000,1.80000,4.20000,0.998602


In [32]:
df1 = df_design.groupby(['yoffset_layer', 'yoffset_plane', 'zshift_layer', 'zshift_plane'], as_index=False)['objective'].mean()
df2 = pd.read_csv('observations/interesting_points.csv').groupby(['yoffset_layer', 'yoffset_plane', 'zshift_layer', 'zshift_plane'], as_index=False)['objective'].mean()

In [33]:
df2

,yoffset_layer,yoffset_plane,zshift_layer,zshift_plane,objective
0,1.80000,0.90000,1.80000,4.20000,0.999773
1,1.80000,1.20000,1.80000,4.40000,1.000000
2,1.81362,0.96872,1.98391,5.93714,0.999881
3,1.90000,1.30000,1.60000,4.20000,0.999443
4,1.90000,1.30000,1.80000,4.20000,0.999338


In [34]:
df1

,yoffset_layer,yoffset_plane,zshift_layer,zshift_plane,objective
0,1.80000,0.90000,1.80000,4.20000,0.999800
1,1.80000,1.20000,1.80000,4.40000,1.000000
2,1.81362,0.96872,1.98391,5.93714,0.999808
3,1.90000,1.30000,1.60000,4.20000,0.999045
4,1.90000,1.30000,1.80000,4.20000,0.998602


In [44]:
df = df1
df['objective'] = (df1.objective * 3 + df3.objective * 3 + df2.objective * 5) / 11

In [45]:
df

,yoffset_layer,yoffset_plane,zshift_layer,zshift_plane,objective
0,1.80000,0.90000,1.80000,4.20000,0.999838
1,1.80000,1.20000,1.80000,4.40000,1.000000
2,1.81362,0.96872,1.98391,5.93714,0.999852
3,1.90000,1.30000,1.60000,4.20000,0.999457
4,1.90000,1.30000,1.80000,4.20000,0.999340


In [47]:
1100 * 2 * 0.999838

2199.6436